In [1]:
import pickle
import time
from typing import List

import numpy as np

from quara.data_analyzer import data_analyzer
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

In [27]:
def estimate(name: str, true_object: State, num_data: List[int], iterations: int, on_para_eq_constraint: bool=True):
    qst = StandardQst(povms, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    obj_sequences = []
    start = time.time()
    for iteration in range(iterations):
        seeds = [iteration] * len(num_data)
        empi_dists_seq = qst.generate_empi_dists_sequence(
            true_object, num_data, seeds
        )

        estimator = LinearEstimator()
        #obj_sequence = estimator.calc_estimate_sequence_qoperation(qst, empi_dists_seq)
        obj_sequence = estimator.calc_estimate_sequence(qst, empi_dists_seq)

        info = {
            "iteration": iteration,
            "empi_dists_seq": empi_dists_seq,
            "obj_sequence": obj_sequence.estimated_var_sequence,
        }
        print(info)
        obj_sequences.append(obj_sequence.estimated_qoperation_sequence)

    end = time.time()
    print(f"time(s)={end - start}")
    #with open(f"state_obj_sequences_{name}_{on_para_eq_constraint}_{iterations}.pkl", "wb") as f:
    #    pickle.dump(obj_sequences, f)

    #with open("obj_sequences.pkl", "rb") as f:
    #    obj_sequences = pickle.load(f)
    return obj_sequences

In [2]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [50]:
true_object = get_z0_1q(c_sys)
num_data = [100, 1000, 10000, 100000]
iterations = 10

param_affine_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=False)

{'iteration': 0, 'empi_dists_seq': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.517, 0.483])), (1000, array([0.517, 0.483])), (1000, array([1., 0.]))], [(10000, array([0.5064, 0.4936])), (10000, array([0.5064, 0.4936])), (10000, array([1., 0.]))], [(100000, array([0.50055, 0.49945])), (100000, array([0.50055, 0.49945])), (100000, array([1., 0.]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E130EDD400>, <quara.objects.state.State object at 0x000001E130EDD780>, <quara.objects.state.State object at 0x000001E130EDDBE0>, <quara.objects.state.State object at 0x000001E130EDD5F8>]}
{'iteration': 1, 'empi_dists_seq': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.494, 0.506])), (1000, array([0.494, 0.506])), (1000, array([1., 0.]))], [(10000, array([0.499, 0.501])), (10000, array([0.499, 0.501])), (10000, array([1., 0.]))], [(100000, array([0.50014, 0.49986])), (10000

In [28]:
#### 1
true_object = get_z0_1q(c_sys)
num_data = [100, 1000, 10000, 100000]
iterations = 10

param_affine_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=False)

# calc mse(param_affine)
param_affine_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_affine_est_linear)]
mses_param_affine_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
mses_param_affine_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
print(f"param=affine, est=linear: {mses_param_affine_est_linear}")
print(f"param=affine, est=proj  : {mses_param_affine_est_proj}")

# calc mse(param_linear)
param_linear_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_linear_est_linear)]
mses_param_linear_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
mses_param_linear_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
print(f"param=linear, est=linear: {mses_param_linear_est_linear}")
print(f"param=linear, est=proj  : {mses_param_linear_est_proj}")

{'iteration': 0, 'empi_dists_seq': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.517, 0.483])), (1000, array([0.517, 0.483])), (1000, array([1., 0.]))], [(10000, array([0.5064, 0.4936])), (10000, array([0.5064, 0.4936])), (10000, array([1., 0.]))], [(100000, array([0.50055, 0.49945])), (100000, array([0.50055, 0.49945])), (100000, array([1., 0.]))]], 'obj_sequence': [array([0.01414214, 0.01414214, 0.70710678]), array([0.02404163, 0.02404163, 0.70710678]), array([0.00905097, 0.00905097, 0.70710678]), array([0.00077782, 0.00077782, 0.70710678])]}
{'iteration': 1, 'empi_dists_seq': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.494, 0.506])), (1000, array([0.494, 0.506])), (1000, array([1., 0.]))], [(10000, array([0.499, 0.501])), (10000, array([0.499, 0.501])), (10000, array([1., 0.]))], [(100000, array([0.50014, 0.49986])), (100000, array([0.50014, 0.49986])), (100000, array([1., 0

In [33]:
##### 2
from quara.data_analyzer import data_analyzer
true_object = get_z0_1q(c_sys)
num_data = [100, 1000, 10000, 100000]
iterations = 10
estimator = LinearEstimator()

param_affine_est_linear = data_analyzer.calc_estimate(povms, true_object, num_data, iterations, estimator, on_para_eq_constraint=True)
param_linear_est_linear = data_analyzer.calc_estimate(povms, true_object, num_data, iterations, estimator, on_para_eq_constraint=False)

param_affine_est_linear = [result.estimated_qoperation_sequence for result in param_affine_est_linear]
param_linear_est_linear = [result.estimated_qoperation_sequence for result in param_linear_est_linear]

# calc mse(param_affine)
#param_affine_est_linear_tmp = [obj_sequence.estimated_qoperation_sequence for obj_sequence in zip(*param_affine_est_linear)]
param_affine_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_affine_est_linear)]
mses_param_affine_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
mses_param_affine_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
print(f"param=affine, est=linear: {mses_param_affine_est_linear}")
print(f"param=affine, est=proj  : {mses_param_affine_est_proj}")

# calc mse(param_linear)
param_linear_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_linear_est_linear)]
mses_param_linear_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
mses_param_linear_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
print(f"param=linear, est=linear: {mses_param_linear_est_linear}")
print(f"param=linear, est=proj  : {mses_param_linear_est_proj}")

{'iteration': 1, 'data': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.517, 0.483])), (1000, array([0.517, 0.483])), (1000, array([1., 0.]))], [(10000, array([0.5064, 0.4936])), (10000, array([0.5064, 0.4936])), (10000, array([1., 0.]))], [(100000, array([0.50055, 0.49945])), (100000, array([0.50055, 0.49945])), (100000, array([1., 0.]))]], 'estimated_var_sequence': [array([0.01414214, 0.01414214, 0.70710678]), array([0.02404163, 0.02404163, 0.70710678]), array([0.00905097, 0.00905097, 0.70710678]), array([0.00077782, 0.00077782, 0.70710678])], 'computation_times': [0.0, 0.0, 0.0, 0.0]}
{'iteration': 2, 'data': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.494, 0.506])), (1000, array([0.494, 0.506])), (1000, array([1., 0.]))], [(10000, array([0.499, 0.501])), (10000, array([0.499, 0.501])), (10000, array([1., 0.]))], [(100000, array([0.50014, 0.49986])), (100000, array([0.50014, 

In [40]:
##### 3
from quara.data_analyzer import data_analyzer
from quara.protocol.qtomography.standard.standard_qtomography_estimator import (
    StandardQTomographyEstimator,
    StandardQTomographyEstimationResult,
)

def convert_to_series(
    results: List[StandardQTomographyEstimationResult], true_object: QOperation
):
    # calc mse
    results_tmp = [result.estimated_qoperation_sequence for result in results]
    results_tmp = [list(qoperation_sequence) for qoperation_sequence in zip(*results_tmp)]
    mses = [calc_mse(qoperation_sequence, [true_object] * len(qoperation_sequence)) for qoperation_sequence in results_tmp]

    # convert to computation time series
    comp_time_tmp = [result.computation_times for result in results]
    comp_time = [list(comp_time) for comp_time in zip(*comp_time_tmp)]

    return mses, comp_time

true_object = get_z0_1q(c_sys)
num_data = [100, 1000, 10000, 100000]
iterations = 10
estimator = LinearEstimator()

# calculate
results_affine_est_linear = data_analyzer.calc_estimate(povms, true_object, num_data, iteration, estimator, on_para_eq_constraint=True)
param_affine_est_linear, time_affine_est_linear = convert_to_series(results_affine_est_linear, true_object)

#results_linear_est_linear = data_analyzer.calc_estimate(povms, true_object, num_data, iteration, estimator, on_para_eq_constraint=False)
#param_linear_est_linear, time_linear_est_linear = convert_to_series(results_linear_est_linear, true_object)

print(f"param=linear, est=linear: {param_affine_est_linear}")
#print(f"param=linear, est=proj  : {param_linear_est_linear}")

{'iteration': 1, 'data': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.517, 0.483])), (1000, array([0.517, 0.483])), (1000, array([1., 0.]))], [(10000, array([0.5064, 0.4936])), (10000, array([0.5064, 0.4936])), (10000, array([1., 0.]))], [(100000, array([0.50055, 0.49945])), (100000, array([0.50055, 0.49945])), (100000, array([1., 0.]))]], 'estimated_var_sequence': [array([0.01414214, 0.01414214, 0.70710678]), array([0.02404163, 0.02404163, 0.70710678]), array([0.00905097, 0.00905097, 0.70710678]), array([0.00077782, 0.00077782, 0.70710678])], 'computation_times': [0.0, 0.0, 0.0, 0.0]}
{'iteration': 2, 'data': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.494, 0.506])), (1000, array([0.494, 0.506])), (1000, array([1., 0.]))], [(10000, array([0.499, 0.501])), (10000, array([0.499, 0.501])), (10000, array([1., 0.]))], [(100000, array([0.50014, 0.49986])), (100000, array([0.50014, 

In [24]:
##### 4
vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)
#true_object = get_z0_1q(c_sys)
num_data = [100, 1000, 10000, 100000]
iteration = 100
#estimator = LinearEstimator()
estimator = ProjectedLinearEstimator()

# calculate
results_affine_est_linear = data_analyzer.calc_estimate(povms, true_object, num_data, iteration, estimator, on_para_eq_constraint=True)
mse_affine_est_linear, std_affine_est_linear, time_affine_est_linear = convert_to_series(results_affine_est_linear, true_object)

#results_linear_est_linear = data_analyzer.calc_estimate(povms, true_object, num_data, iteration, estimator, on_para_eq_constraint=False)
#mse_linear_est_linear, std_linear_est_linear, time_linear_est_linear = data_analyzer.convert_to_series(results_linear_est_linear, true_object)

#print(f"param=linear, est=linear: {param_affine_est_linear}")
#print(f"param=linear, est=proj  : {param_linear_est_linear}")

mes': [0.0030007362365722656, 0.00099945068359375, 0.0, 0.00099945068359375]}
{'iteration': 74, 'data': [[(100, array([0.79, 0.21])), (100, array([0.79, 0.21])), (100, array([0.79, 0.21]))], [(1000, array([0.782, 0.218])), (1000, array([0.782, 0.218])), (1000, array([0.782, 0.218]))], [(10000, array([0.7817, 0.2183])), (10000, array([0.7817, 0.2183])), (10000, array([0.7817, 0.2183]))], [(100000, array([0.78785, 0.21215])), (100000, array([0.78785, 0.21215])), (100000, array([0.78785, 0.21215]))]], 'estimated_var_sequence': [array([0.40918511, 0.40918511, 0.40918511]), array([0.39880822, 0.39880822, 0.39880822]), array([0.39838396, 0.39838396, 0.39838396]), array([0.40708137, 0.40708137, 0.40708137])], 'computation_times': [0.0009980201721191406, 0.0, 0.0010113716125488281, 0.00299072265625]}
{'iteration': 75, 'data': [[(100, array([0.85, 0.15])), (100, array([0.85, 0.15])), (100, array([0.85, 0.15]))], [(1000, array([0.785, 0.215])), (1000, array([0.785, 0.215])), (1000, array([0.785,

In [25]:
### now!
import plotly.graph_objects as go
from quara.protocol.qtomography.standard.standard_qtomography_estimator import (
    StandardQTomographyEstimator,
    StandardQTomographyEstimationResult,
)

# common
def calc_mse(xs: List[QOperation], ys: List[QOperation]) -> np.float64:
    points = []
    for x, y in zip(xs, ys):
        x_vec = x.to_stacked_vector()
        y_vec = y.to_stacked_vector()
        point = np.vdot(x_vec - y_vec, x_vec - y_vec)
        points.append(point)
        print(point, x_vec)

    mse = np.mean(points, dtype=np.float64)
    std = np.std(points, dtype=np.float64, ddof=1)
    return mse, std

# common(StandardQTomography)
def convert_to_series(
    results: List[StandardQTomographyEstimationResult], true_object: QOperation
):
    # calc mse
    results_tmp = [result.estimated_qoperation_sequence for result in results]
    results_tmp = [
        list(qoperation_sequence) for qoperation_sequence in zip(*results_tmp)
    ]
    mses = [
        calc_mse(qoperation_sequence, [true_object] * len(qoperation_sequence))
        for qoperation_sequence in results_tmp
    ]
    stds = [mse[1] for mse in mses]
    mses = [mse[0] for mse in mses]

    # convert to computation time series
    comp_time_tmp = [result.computation_times for result in results]
    comp_time = [list(comp_time) for comp_time in zip(*comp_time_tmp)]

    return mses, stds, comp_time

# common(show log-log scatter?)
def show_mse(name: str, num_data: List[int], mses: List[float], std_mses: List[float], num_of_estimates: int, title: str = None):
    if title is None:
        title = f"QST<br> num of estimates={num_of_estimates}"
    trace = go.Scatter(
        name=name,
        x=num_data,
        y=mses,
        mode="lines+markers",
        error_y = dict(
            type="data",
            array=std_mses,
            visible=True
        ),
    )
    data = [trace]
    layout = go.Layout(
        title=title,
        xaxis_title_text="number of data",
        yaxis_title_text="Mean Square Error of estimates and true",
        xaxis_type="log",
        yaxis_type="log",
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()


# show graphs
#print(param_affine_est_linear)
#param_affine_est_linear_mses, param_affine_est_linear_stds, time_affine_est_linear = convert_to_series(results_affine_est_linear, true_object)
show_mse("param=affine, est=linear", num_data, mse_affine_est_linear, std_affine_est_linear, iteration)
#data_analyzer.show_mse(num_data, param_affine_est_linear)
print(mse_affine_est_linear)
print(std_affine_est_linear)

[0.0037198670102283637, 0.00038518391491347727, 5.520930649354862e-05, 5.2654284573857005e-06]
[0.008957848102373949, 0.000924632500317718, 9.79191411262894e-05, 6.875151067797309e-06]


In [38]:
for index in range(100):
    print(results_affine_est_linear[index].estimated_var_sequence[0])

[0.40836244 0.40836244 0.40836244]
[0.40874878 0.40874878 0.40874878]
[0.40833298 0.40833298 0.40833298]
[0.40830445 0.40830445 0.40830445]
[0.40874878 0.40874878 0.40874878]
[0.33941125 0.33941125 0.33941125]
[0.40874878 0.40874878 0.40874878]
[0.4084839 0.4084839 0.4084839]
[0.40874878 0.40874878 0.40874878]
[0.4084839 0.4084839 0.4084839]
[0.40842134 0.40842134 0.40842134]
[0.40874878 0.40874878 0.40874878]
[0.38183766 0.38183766 0.38183766]
[0.40874878 0.40874878 0.40874878]
[0.40834679 0.40834679 0.40834679]
[0.40839006 0.40839006 0.40839006]
[0.40842134 0.40842134 0.40842134]
[0.38183766 0.38183766 0.38183766]
[0.3959798 0.3959798 0.3959798]
[0.40918511 0.40918511 0.40918511]
[0.40839006 0.40839006 0.40839006]
[0.35355339 0.35355339 0.35355339]
[0.3959798 0.3959798 0.3959798]
[0.36769553 0.36769553 0.36769553]
[0.32526912 0.32526912 0.32526912]
[0.40836244 0.40836244 0.40836244]
[0.38183766 0.38183766 0.38183766]
[0.32526912 0.32526912 0.32526912]
[0.33941125 0.33941125 0.3394112

In [33]:
print(0.40874878**2 + 0.40874878**2 + 0.40874878**2)
print(0.33941125**2 + 0.33941125**2 + 0.33941125**2)

0.5012266954544652
0.3455999898796875


In [7]:
# common(show log-log scatter?)
def show_mse(num_data: List[int], mses: List[float], std_mses: List[float], title: str = "Mean Square Value"):
    trace = go.Scatter(
        x=num_data,
        y=mses,
        mode="lines+markers",
        error_y = dict(
            type="data",
            array=std_mses,
            visible=True
        ),
    )
    data = [trace]
    max_value = np.max(mses) + np.max(std_mses)
    layout = go.Layout(
        title=title,
        xaxis_title_text="number of data",
        yaxis_title_text="Mean Square Error of estimates and true",
        yaxis_range=[0, max_value * 1.1],
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()

show_mse([10,20,30], [50, 60, 70], [5, 5, 5])

In [8]:
print(0.01413931**2 +  0.01413931**2 +  0.70696544**2)
print(0.08425476**2 +  0.08425476**2 +  0.70212301**2)

0.5001999735289457
0.5071744503367753


In [41]:
x = [0.00040000000000000186, 0.00040000000000000186, 0.014400000000000015, 0.0036000000000000073, 0.014400000000000015, 0.00040000000000000186, 0.010000000000000012, 0.0016000000000000044, 0.0016000000000000044, 0.014400000000000015]
np.mean(x)

0.006120000000000008

In [31]:
from quara.protocol.qtomography.standard.standard_qtomography_estimator import (
    StandardQTomographyEstimator,
    StandardQTomographyEstimationResult,
)

# common
def calc_mse(xs: List[QOperation], ys: List[QOperation]) -> np.float64:
    points = []
    for x, y in zip(xs, ys):
        x_vec = x.to_stacked_vector()
        y_vec = y.to_stacked_vector()
        point = np.vdot(x_vec - y_vec, x_vec - y_vec)
        points.append(point)

    mse = np.mean(points, dtype=np.float64)
    std = np.std(points, dtype=np.float64, ddof=1)
    return mse, std

def convert_to_series(
    results: List[StandardQTomographyEstimationResult], true_object: QOperation
):
    # calc mse
    results_tmp = [result.estimated_qoperation_sequence for result in results]
    results_tmp = [
        list(qoperation_sequence) for qoperation_sequence in zip(*results_tmp)
    ]
    mses_stds = [
        calc_mse(qoperation_sequence, [true_object] * len(qoperation_sequence))
        for qoperation_sequence in results_tmp
    ]

    # convert to computation time series
    comp_time_tmp = [result.computation_times for result in results]
    comp_time = [list(comp_time) for comp_time in zip(*comp_time_tmp)]

    return mses_stds, comp_time

param_affine_est_linear2, time_affine_est_linear = convert_to_series(results_affine_est_linear, true_object)
print(param_affine_est_linear2)
#data_analyzer.show_mse(num_data, param_affine_est_linear)
show_mse(num_data, param_affine_est_linear)

0.006377878435132909
0.0013753888177530022
0.00021323092808814848
1.4750693705781257e-05


TypeError: show_mse() missing 1 required positional argument: 'std_mses'

In [51]:
# calc mse(param_affine)
param_affine_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_affine_est_linear)]
mses_param_affine_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
mses_param_affine_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
print(f"param=affine, est=linear: {mses_param_affine_est_linear}")
print(f"param=affine, est=proj  : {mses_param_affine_est_proj}")

# calc mse(param_linear)
param_linear_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_linear_est_linear)]
mses_param_linear_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
mses_param_linear_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
print(f"param=linear, est=linear: {mses_param_linear_est_linear}")
print(f"param=linear, est=proj  : {mses_param_linear_est_proj}")

param=affine, est=linear: [0.006120000000000008, 0.0009372000000000015, 0.00014730399999999952, 9.04508000000003e-06]
param=affine, est=proj  : [0.006056538960553601, 0.000934955923290004, 0.00014724925905300592, 9.044837081659776e-06]
param=linear, est=linear: [0.006120000000000007, 0.0009372000000000023, 0.0001473039999999987, 9.04507999999998e-06]
param=linear, est=proj  : [0.006056538960553604, 0.0009349559232900045, 0.0001472492590530051, 9.044837081659727e-06]


In [52]:
import plotly.graph_objects as go

data_param_affine_est_linear = go.Scatter(name='param=affine, est=linear', x=num_data, y=mses_param_affine_est_linear, mode='lines+markers')
data_param_affine_est_proj = go.Scatter(name='param=affine, est=proj', x=num_data, y=mses_param_affine_est_proj, mode='lines+markers')
data_param_linear_est_linear = go.Scatter(name='param=linear, est=linear', x=num_data, y=mses_param_linear_est_linear, mode='lines+markers')
data_param_linear_est_proj = go.Scatter(name='param=linear, est=proj', x=num_data, y=mses_param_linear_est_proj, mode='lines+markers')
data = [data_param_affine_est_linear, data_param_affine_est_proj, data_param_linear_est_linear, data_param_linear_est_proj]

layout = go.Layout(title='QST', xaxis_title_text="number of data", yaxis_title_text="Mean Square Error of estimates and true", xaxis_type="log", yaxis_type="log")
fig = go.Figure(data=data, layout=layout)
fig.show()

# 真値をz0 = [ 1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6) ]

In [56]:
vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [100, 1000, 10000, 100000]
iterations = 10

param_affine_est_linear = estimate("sqrt6", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("sqrt6", true_object, num_data, iterations, on_para_eq_constraint=False)

{'iteration': 0, 'empi_dists_seq': [[(100, array([0.83, 0.17])), (100, array([0.83, 0.17])), (100, array([0.83, 0.17]))], [(1000, array([0.787, 0.213])), (1000, array([0.787, 0.213])), (1000, array([0.787, 0.213]))], [(10000, array([0.791, 0.209])), (10000, array([0.791, 0.209])), (10000, array([0.791, 0.209]))], [(100000, array([0.78824, 0.21176])), (100000, array([0.78824, 0.21176])), (100000, array([0.78824, 0.21176]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E131DF8FD0>, <quara.objects.state.State object at 0x000001E131C74940>, <quara.objects.state.State object at 0x000001E131C74128>, <quara.objects.state.State object at 0x000001E131C740B8>]}
{'iteration': 1, 'empi_dists_seq': [[(100, array([0.8, 0.2])), (100, array([0.8, 0.2])), (100, array([0.8, 0.2]))], [(1000, array([0.791, 0.209])), (1000, array([0.791, 0.209])), (1000, array([0.791, 0.209]))], [(10000, array([0.793, 0.207])), (10000, array([0.793, 0.207])), (10000, array([0.793, 0.207]))], [(100000, ar

In [59]:
# calc mse(param_affine)
param_affine_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_affine_est_linear)]
mses_param_affine_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
mses_param_affine_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
print(f"param=affine, est=linear: {mses_param_affine_est_linear}")
print(f"param=affine, est=proj  : {mses_param_affine_est_proj}")

# calc mse(param_linear)
param_linear_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_linear_est_linear)]
mses_param_linear_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
mses_param_linear_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
print(f"param=linear, est=linear: {mses_param_linear_est_linear}")
print(f"param=linear, est=proj  : {mses_param_linear_est_proj}")

param=affine, est=linear: [0.009524397692158331, 0.00036800967188586626, 8.490209459249662e-05, 9.130723205515926e-06]
param=affine, est=proj  : [0.001421902142277602, 9.340498882317307e-05, 3.4230038712040195e-05, 5.942278830469556e-06]
param=linear, est=linear: [0.009524397692158317, 0.0003680096718858644, 8.49020945924957e-05, 9.130723205516e-06]
param=linear, est=proj  : [0.001421902142277593, 9.340498882317226e-05, 3.423003871204143e-05, 5.942278830469444e-06]


In [58]:
import plotly.graph_objects as go

data_param_affine_est_linear = go.Scatter(name='param=affine, est=linear', x=num_data, y=mses_param_affine_est_linear, mode='lines+markers')
data_param_affine_est_proj = go.Scatter(name='param=affine, est=proj', x=num_data, y=mses_param_affine_est_proj, mode='lines+markers')
data_param_linear_est_linear = go.Scatter(name='param=linear, est=linear', x=num_data, y=mses_param_linear_est_linear, mode='lines+markers')
data_param_linear_est_proj = go.Scatter(name='param=linear, est=proj', x=num_data, y=mses_param_linear_est_proj, mode='lines+markers')
data = [data_param_affine_est_linear, data_param_affine_est_proj, data_param_linear_est_linear, data_param_linear_est_proj]

layout = go.Layout(title='QST', xaxis_title_text="number of data", yaxis_title_text="Mean Square Error of estimates and true", xaxis_type="log", yaxis_type="log")
fig = go.Figure(data=data, layout=layout)
fig.show()

# # 40回。真値を[ 1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6) ]

In [60]:
vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [100, 1000, 10000, 100000]
iterations = 40

param_affine_est_linear = estimate("sqrt6", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("sqrt6", true_object, num_data, iterations, on_para_eq_constraint=False)

ject at 0x000001E131CE42E8>, <quara.objects.state.State object at 0x000001E131A87240>, <quara.objects.state.State object at 0x000001E131CD8D30>]}
{'iteration': 11, 'empi_dists_seq': [[(100, array([0.8, 0.2])), (100, array([0.8, 0.2])), (100, array([0.8, 0.2]))], [(1000, array([0.773, 0.227])), (1000, array([0.773, 0.227])), (1000, array([0.773, 0.227]))], [(10000, array([0.7904, 0.2096])), (10000, array([0.7904, 0.2096])), (10000, array([0.7904, 0.2096]))], [(100000, array([0.78816, 0.21184])), (100000, array([0.78816, 0.21184])), (100000, array([0.78816, 0.21184]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E131CD8390>, <quara.objects.state.State object at 0x000001E131CD8C88>, <quara.objects.state.State object at 0x000001E131CD8F60>, <quara.objects.state.State object at 0x000001E131CD8550>]}
{'iteration': 12, 'empi_dists_seq': [[(100, array([0.77, 0.23])), (100, array([0.77, 0.23])), (100, array([0.77, 0.23]))], [(1000, array([0.767, 0.233])), (1000, array([0.767

In [61]:
# calc mse(param_affine)
param_affine_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_affine_est_linear)]
mses_param_affine_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
mses_param_affine_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
print(f"param=affine, est=linear: {mses_param_affine_est_linear}")
print(f"param=affine, est=proj  : {mses_param_affine_est_proj}")

# calc mse(param_linear)
param_linear_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_linear_est_linear)]
mses_param_linear_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
mses_param_linear_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
print(f"param=linear, est=linear: {mses_param_linear_est_linear}")
print(f"param=linear, est=proj  : {mses_param_linear_est_proj}")

param=affine, est=linear: [0.010798226764637906, 0.0006778246788129034, 0.00011081240423948502, 1.021381388129537e-05]
param=affine, est=proj  : [0.005098065091323841, 0.0003912358509350632, 6.453912453460148e-05, 6.419799405647e-06]
param=linear, est=linear: [0.01079822676463791, 0.0006778246788129045, 0.00011081240423948509, 1.0213813881295394e-05]
param=linear, est=proj  : [0.005098065091323858, 0.0003912358509350642, 6.453912453460236e-05, 6.419799405647193e-06]


In [62]:
import plotly.graph_objects as go

data_param_affine_est_linear = go.Scatter(name='param=affine, est=linear', x=num_data, y=mses_param_affine_est_linear, mode='lines+markers')
data_param_affine_est_proj = go.Scatter(name='param=affine, est=proj', x=num_data, y=mses_param_affine_est_proj, mode='lines+markers')
data_param_linear_est_linear = go.Scatter(name='param=linear, est=linear', x=num_data, y=mses_param_linear_est_linear, mode='lines+markers')
data_param_linear_est_proj = go.Scatter(name='param=linear, est=proj', x=num_data, y=mses_param_linear_est_proj, mode='lines+markers')
data = [data_param_affine_est_linear, data_param_affine_est_proj, data_param_linear_est_linear, data_param_linear_est_proj]

layout = go.Layout(title='QST', xaxis_title_text="number of data", yaxis_title_text="Mean Square Error of estimates and true", xaxis_type="log", yaxis_type="log")
fig = go.Figure(data=data, layout=layout)
fig.show()

# デバッグ

In [66]:
qst_affine = StandardQst(povms, on_para_eq_constraint=True)
print("-- affine --")
print("matA=")
print(qst_affine.calc_matA())
print("vecB=")
print(qst_affine.calc_vecB())

-- affine --
matA=
[[ 0.70710678  0.          0.        ]
 [-0.70710678  0.          0.        ]
 [ 0.          0.70710678  0.        ]
 [ 0.         -0.70710678  0.        ]
 [ 0.          0.          0.70710678]
 [ 0.          0.         -0.70710678]]
vecB=
[0.70710678 0.70710678 0.70710678 0.70710678 0.70710678 0.70710678]


In [67]:
qst_linear = StandardQst(povms, on_para_eq_constraint=False)
print("-- linear --")
print("matA=")
print(qst_linear.calc_matA())
print("vecB=")
print(qst_linear.calc_vecB())

-- linear --
matA=
[[ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.70710678  0.          0.70710678  0.        ]
 [ 0.70710678  0.         -0.70710678  0.        ]
 [ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]
vecB=
[0 0 0 0 0 0]


# 本番

In [68]:
true_object = get_z0_1q(c_sys)
num_data = [100, 1000, 10000, 100000]
iterations = 1000

param_affine_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=False)

 0.]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E131ADB278>, <quara.objects.state.State object at 0x000001E131ADB438>, <quara.objects.state.State object at 0x000001E131ADBDD8>, <quara.objects.state.State object at 0x000001E131ADB2E8>]}
{'iteration': 970, 'empi_dists_seq': [[(100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.495, 0.505])), (1000, array([0.495, 0.505])), (1000, array([1., 0.]))], [(10000, array([0.4984, 0.5016])), (10000, array([0.4984, 0.5016])), (10000, array([1., 0.]))], [(100000, array([0.50024, 0.49976])), (100000, array([0.50024, 0.49976])), (100000, array([1., 0.]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E131ADB4A8>, <quara.objects.state.State object at 0x000001E131ADB630>, <quara.objects.state.State object at 0x000001E131ADB8D0>, <quara.objects.state.State object at 0x000001E131ADB9B0>]}
{'iteration': 971, 'empi_dists_seq': [[(100, array([0.49, 0.51])), (100, array([0

In [69]:
# calc mse(param_affine)
param_affine_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_affine_est_linear)]
mses_param_affine_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
mses_param_affine_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
print(f"param=affine, est=linear: {mses_param_affine_est_linear}")
print(f"param=affine, est=proj  : {mses_param_affine_est_proj}")

# calc mse(param_linear)
param_linear_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_linear_est_linear)]
mses_param_linear_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
mses_param_linear_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
print(f"param=linear, est=linear: {mses_param_linear_est_linear}")
print(f"param=linear, est=proj  : {mses_param_linear_est_proj}")

param=affine, est=linear: [0.011150400000000008, 0.0010314760000000019, 0.00010338396000000001, 1.022362080000001e-05]
param=affine, est=proj  : [0.01070085428640879, 0.001028742115271585, 0.0001033545699697992, 1.0223345250594405e-05]
param=linear, est=linear: [0.011150400000000005, 0.0010314760000000019, 0.00010338395999999994, 1.0223620799999999e-05]
param=linear, est=proj  : [0.010700854286408788, 0.0010287421152715853, 0.00010335456996979914, 1.0223345250594398e-05]


In [70]:
import plotly.graph_objects as go

data_param_affine_est_linear = go.Scatter(name='param=affine, est=linear', x=num_data, y=mses_param_affine_est_linear, mode='lines+markers')
data_param_affine_est_proj = go.Scatter(name='param=affine, est=proj', x=num_data, y=mses_param_affine_est_proj, mode='lines+markers')
data_param_linear_est_linear = go.Scatter(name='param=linear, est=linear', x=num_data, y=mses_param_linear_est_linear, mode='lines+markers')
data_param_linear_est_proj = go.Scatter(name='param=linear, est=proj', x=num_data, y=mses_param_linear_est_proj, mode='lines+markers')
data = [data_param_affine_est_linear, data_param_affine_est_proj, data_param_linear_est_linear, data_param_linear_est_proj]

layout = go.Layout(title='QST', xaxis_title_text="number of data", yaxis_title_text="Mean Square Error of estimates and true", xaxis_type="log", yaxis_type="log")
fig = go.Figure(data=data, layout=layout)
fig.show()

# State([ 1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6) ]) , 反復回数100回

In [71]:
vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [100, 1000, 10000, 100000]
iterations = 100

param_affine_est_linear = estimate("sqrt6", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("sqrt6", true_object, num_data, iterations, on_para_eq_constraint=False)

#####
# calc mse(param_affine)
param_affine_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_affine_est_linear)]
mses_param_affine_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
mses_param_affine_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
print(f"param=affine, est=linear: {mses_param_affine_est_linear}")
print(f"param=affine, est=proj  : {mses_param_affine_est_proj}")

# calc mse(param_linear)
param_linear_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_linear_est_linear)]
mses_param_linear_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
mses_param_linear_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
print(f"param=linear, est=linear: {mses_param_linear_est_linear}")
print(f"param=linear, est=proj  : {mses_param_linear_est_proj}")

#####
data_param_affine_est_linear = go.Scatter(name='param=affine, est=linear', x=num_data, y=mses_param_affine_est_linear, mode='lines+markers')
data_param_affine_est_proj = go.Scatter(name='param=affine, est=proj', x=num_data, y=mses_param_affine_est_proj, mode='lines+markers')
data_param_linear_est_linear = go.Scatter(name='param=linear, est=linear', x=num_data, y=mses_param_linear_est_linear, mode='lines+markers')
data_param_linear_est_proj = go.Scatter(name='param=linear, est=proj', x=num_data, y=mses_param_linear_est_proj, mode='lines+markers')
data = [data_param_affine_est_linear, data_param_affine_est_proj, data_param_linear_est_linear, data_param_linear_est_proj]

layout = go.Layout(title='QST', xaxis_title_text="number of data", yaxis_title_text="Mean Square Error of estimates and true", xaxis_type="log", yaxis_type="log")
fig = go.Figure(data=data, layout=layout)
fig.show()

ay([0.78797, 0.21203])), (100000, array([0.78797, 0.21203])), (100000, array([0.78797, 0.21203]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E13065EE48>, <quara.objects.state.State object at 0x000001E13065EEF0>, <quara.objects.state.State object at 0x000001E13065E9E8>, <quara.objects.state.State object at 0x000001E13065ECC0>]}
{'iteration': 72, 'empi_dists_seq': [[(100, array([0.86, 0.14])), (100, array([0.86, 0.14])), (100, array([0.86, 0.14]))], [(1000, array([0.789, 0.211])), (1000, array([0.789, 0.211])), (1000, array([0.789, 0.211]))], [(10000, array([0.7884, 0.2116])), (10000, array([0.7884, 0.2116])), (10000, array([0.7884, 0.2116]))], [(100000, array([0.78689, 0.21311])), (100000, array([0.78689, 0.21311])), (100000, array([0.78689, 0.21311]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E133055048>, <quara.objects.state.State object at 0x000001E131C60D68>, <quara.objects.state.State object at 0x000001E131C60320>, <quara.objects.state.S

# State([ 1/np.sqrt(2), 0.99/np.sqrt(6), 0.99/np.sqrt(6), 0.99/np.sqrt(6) ]), 反復回数100回


In [72]:
vec = np.array([1/np.sqrt(2), 0.99/np.sqrt(6), 0.99/np.sqrt(6), 0.99/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [100, 1000, 10000, 100000]
iterations = 100

param_affine_est_linear = estimate("0.99", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("0.99", true_object, num_data, iterations, on_para_eq_constraint=False)

#####
# calc mse(param_affine)
param_affine_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_affine_est_linear)]
mses_param_affine_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
mses_param_affine_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
print(f"param=affine, est=linear: {mses_param_affine_est_linear}")
print(f"param=affine, est=proj  : {mses_param_affine_est_proj}")

# calc mse(param_linear)
param_linear_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_linear_est_linear)]
mses_param_linear_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
mses_param_linear_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
print(f"param=linear, est=linear: {mses_param_linear_est_linear}")
print(f"param=linear, est=proj  : {mses_param_linear_est_proj}")

#####
data_param_affine_est_linear = go.Scatter(name='param=affine, est=linear', x=num_data, y=mses_param_affine_est_linear, mode='lines+markers')
data_param_affine_est_proj = go.Scatter(name='param=affine, est=proj', x=num_data, y=mses_param_affine_est_proj, mode='lines+markers')
data_param_linear_est_linear = go.Scatter(name='param=linear, est=linear', x=num_data, y=mses_param_linear_est_linear, mode='lines+markers')
data_param_linear_est_proj = go.Scatter(name='param=linear, est=proj', x=num_data, y=mses_param_linear_est_proj, mode='lines+markers')
data = [data_param_affine_est_linear, data_param_affine_est_proj, data_param_linear_est_linear, data_param_linear_est_proj]

layout = go.Layout(title='QST', xaxis_title_text="number of data", yaxis_title_text="Mean Square Error of estimates and true", xaxis_type="log", yaxis_type="log")
fig = go.Figure(data=data, layout=layout)
fig.show()

[0.781, 0.219]))], [(100000, array([0.78492, 0.21508])), (100000, array([0.78492, 0.21508])), (100000, array([0.78492, 0.21508]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E131AE3F28>, <quara.objects.state.State object at 0x000001E131AE32B0>, <quara.objects.state.State object at 0x000001E131AE3F60>, <quara.objects.state.State object at 0x000001E131AE3940>]}
{'iteration': 72, 'empi_dists_seq': [[(100, array([0.85, 0.15])), (100, array([0.85, 0.15])), (100, array([0.85, 0.15]))], [(1000, array([0.786, 0.214])), (1000, array([0.786, 0.214])), (1000, array([0.786, 0.214]))], [(10000, array([0.7863, 0.2137])), (10000, array([0.7863, 0.2137])), (10000, array([0.7863, 0.2137]))], [(100000, array([0.78417, 0.21583])), (100000, array([0.78417, 0.21583])), (100000, array([0.78417, 0.21583]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E131AE3198>, <quara.objects.state.State object at 0x000001E131AE33C8>, <quara.objects.state.State object at 0x000001E13

# State([ 1/np.sqrt(2), 0.95/np.sqrt(6), 0.95/np.sqrt(6), 0.95/np.sqrt(6) ]), 反復回数100回

In [73]:
vec = np.array([1/np.sqrt(2), 0.95/np.sqrt(6), 0.95/np.sqrt(6), 0.95/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [100, 1000, 10000, 100000]
iterations = 100

param_affine_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("z0", true_object, num_data, iterations, on_para_eq_constraint=False)

#####
# calc mse(param_affine)
param_affine_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_affine_est_linear)]
mses_param_affine_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
mses_param_affine_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_affine_est_linear_tmp
]
print(f"param=affine, est=linear: {mses_param_affine_est_linear}")
print(f"param=affine, est=proj  : {mses_param_affine_est_proj}")

# calc mse(param_linear)
param_linear_est_linear_tmp = [list(obj_sequence) for obj_sequence in zip(*param_linear_est_linear)]
mses_param_linear_est_linear = [
    calc_mse(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
mses_param_linear_est_proj = [
    calc_mse_proj(obj_sequence, [true_object] * len(obj_sequence)) for obj_sequence in param_linear_est_linear_tmp
]
print(f"param=linear, est=linear: {mses_param_linear_est_linear}")
print(f"param=linear, est=proj  : {mses_param_linear_est_proj}")

#####
data_param_affine_est_linear = go.Scatter(name='param=affine, est=linear', x=num_data, y=mses_param_affine_est_linear, mode='lines+markers')
data_param_affine_est_proj = go.Scatter(name='param=affine, est=proj', x=num_data, y=mses_param_affine_est_proj, mode='lines+markers')
data_param_linear_est_linear = go.Scatter(name='param=linear, est=linear', x=num_data, y=mses_param_linear_est_linear, mode='lines+markers')
data_param_linear_est_proj = go.Scatter(name='param=linear, est=proj', x=num_data, y=mses_param_linear_est_proj, mode='lines+markers')
data = [data_param_affine_est_linear, data_param_affine_est_proj, data_param_linear_est_linear, data_param_linear_est_proj]

layout = go.Layout(title='QST', xaxis_title_text="number of data", yaxis_title_text="Mean Square Error of estimates and true", xaxis_type="log", yaxis_type="log")
fig = go.Figure(data=data, layout=layout)
fig.show()

rray([0.77337, 0.22663])), (100000, array([0.77337, 0.22663])), (100000, array([0.77337, 0.22663]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E13184A828>, <quara.objects.state.State object at 0x000001E13184AC18>, <quara.objects.state.State object at 0x000001E13184AEF0>, <quara.objects.state.State object at 0x000001E13184AEB8>]}
{'iteration': 72, 'empi_dists_seq': [[(100, array([0.84, 0.16])), (100, array([0.84, 0.16])), (100, array([0.84, 0.16]))], [(1000, array([0.772, 0.228])), (1000, array([0.772, 0.228])), (1000, array([0.772, 0.228]))], [(10000, array([0.7726, 0.2274])), (10000, array([0.7726, 0.2274])), (10000, array([0.7726, 0.2274]))], [(100000, array([0.77202, 0.22798])), (100000, array([0.77202, 0.22798])), (100000, array([0.77202, 0.22798]))]], 'obj_sequence': [<quara.objects.state.State object at 0x000001E13184A780>, <quara.objects.state.State object at 0x000001E13184AE48>, <quara.objects.state.State object at 0x000001E13184AF98>, <quara.objects.state

# xxxxxx

In [1]:
import numpy as np

from quara.data_analyzer import data_analyzer
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.state import State
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

In [2]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

# settings
vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)], dtype=np.float64)
true_object = State(c_sys, vec, is_physicality_required=False)
num_data = [100, 1000, 10000, 100000]
iteration = 10

In [3]:
estimator = LinearEstimator()

# calculate
results_affine_est_linear = data_analyzer.calc_estimate(povms, true_object, num_data, iteration, estimator, on_para_eq_constraint=True)
param_affine_est_linear, time_affine_est_linear = data_analyzer.convert_to_series(results_affine_est_linear, true_object)

{'iteration': 1, 'data': [[(100, array([0.83, 0.17])), (100, array([0.83, 0.17])), (100, array([0.83, 0.17]))], [(1000, array([0.787, 0.213])), (1000, array([0.787, 0.213])), (1000, array([0.787, 0.213]))], [(10000, array([0.791, 0.209])), (10000, array([0.791, 0.209])), (10000, array([0.791, 0.209]))], [(100000, array([0.78824, 0.21176])), (100000, array([0.78824, 0.21176])), (100000, array([0.78824, 0.21176]))]], 'estimated_var_sequence': [array([0.46669048, 0.46669048, 0.46669048]), array([0.40587929, 0.40587929, 0.40587929]), array([0.41153615, 0.41153615, 0.41153615]), array([0.40763292, 0.40763292, 0.40763292])], 'computation_times': [0.0, 0.0, 0.0, 0.0]}
{'iteration': 2, 'data': [[(100, array([0.8, 0.2])), (100, array([0.8, 0.2])), (100, array([0.8, 0.2]))], [(1000, array([0.791, 0.209])), (1000, array([0.791, 0.209])), (1000, array([0.791, 0.209]))], [(10000, array([0.793, 0.207])), (10000, array([0.793, 0.207])), (10000, array([0.793, 0.207]))], [(100000, array([0.78839, 0.211

In [11]:
from typing import List
import plotly.graph_objects as go

def show_mse(num_data: List[int], mses: List[float], title: str = "Mean Square Value"):
    trace = go.Scatter(x=num_data, y=mses, mode="lines+markers")
    data = [trace]
    print(mses)
    max_value = np.max(mses)
    layout = go.Layout(
        title=title,
        xaxis_title_text="number of data",
        yaxis_title_text="Mean Square Error of estimates and true",
        xaxis_type="log",
        yaxis_type="log",
        #yaxis_range=[0, 0.1],
        #yaxis_range=[0, max_value * 1.1],
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()

# show graphs
show_mse(num_data, param_affine_est_linear)

[0.05910408813987257, 0.06824604305455613, 0.06556260693088675, 0.06726986926873114]
